What is a context?

In Python: the state surrounding a section of code

Context Managers need to:
- create a context (the minimal amount of state needed for a block of code)
- execute some code that uses variables from the context
- automatically clean up the context when we are done with it

Context managers manage the data in our scope, on entry and on exit.

__`try...finally`__

The `finally` section of a try-catch *always* executes. Even if there is an exception or a return statement in the try block. But writing try-except-finally blocks can be cumbersome and visually clutter your code.

__Context Managers__

Described in PEP343. The pattern of context managers is as follows:

```
with context as obj_name:
    # do something with obj_name
    
# after the with block, context is cleaned up automatically
```

### The Context Management Protocol

Classes implement the context management protocol by implementing two methods:
- `__enter__`: setup, and optionally return some object
- `__exit__`: tear down/ cleanup

In [5]:
class CtxManager:
    def __enter__(self):
        return self
    
    def __exit__(self): ...

In [9]:
# we can deconstruct what happens with a context manager here. This is oversimplified.
mgr = CtxManager()
obj = mgr.__enter__()
try:
    #do something with obj
    ...
finally:
    #done w/ context
    mgr.__exit__()

__Use Cases__

Very common usage is for opening/closing a file, but context managers can be used for much more than creating and releasing resources.

Common Patterns:
- Open/Close
- Lock/Release
- Change/Reset
- Start/Stop
- Enter/Exit

__How the Context Protocol Works__

Works in conjunction with the `with` keyword.

`with MyCtxMngr as obj:`
- creates an instance of `MyCtxMngr`, but there is no handle or symbol to this instance.
- calls `ctxmngr_instance.__enter__()`
- return value of `__enter__` is assigned to `obj`, __not__ the instance of `MyctxMngr`
- after the with block, or if an exception occurs within it, `ctxmngr_instance.__exit__()` is called

__Scope of `with` Block__

The `with` block is not like a function or comprehension. The scope of anything in the `with` block, including the object returned from `__enter__` is in the same scope as the `with` statement itself.


In [12]:
class FakeFile:
    def __enter__(self):
        return iter([1, 2, 3, 4])
    
    def __exit__(self, *args):
        return
    
def open(filename: str) -> obj:
    return FakeFile()

In [13]:
with open("fake_file") as f: # f will be stored in the global scope
    row = next(f)  # row will also be in the global scope

In [15]:
# f and row still exist after the with block
print(f)
print(row)

1


**`__enter__`**

Should just perform whatever setup is necessary, it can optionally return an object.

**`__exit__`**

`exit` will always execute, even if an exception occurs (similar to the finally clause of a try-catch). `exit` needs to know about any exceptions that occurred, so it can tell Python to silence the exception or let it propagate.

The `exit` method accepts three arguments:
- the exception type that occurred (None if otherwise)
- the exeception value that occurred (None if otherwise)
- the traceback object if an exception occurred (None if otherwise)

The `exit` method must return a boolean value:
- `True` will silence any raised exception
- `False` will not silence the raised exception


In [16]:
def __exit__(self, exc_type, exc_value, exc_trace):
    return True #or Falseexit

__Caveat with Lazy Iterators__

This will result in an error, since when we try to use the iterator returned by this function, the file object will already be closed by the context manager.
```
def read_data():
    with open('example_csv') as f:
        return csv.reader(f)
```

So we can turn this function into a generator to avoid this bug.
```
def read_data():
    with open('example.csv') as f:
        yield from csv.reader(f)
```

### Generators as Context Managers

__Mimic the Context Manager Pattern using a Generator__

In [15]:
# mask of open for example purposes
def open_(filename):
    class file_obj:
        def close(self): ...
    
    return file_obj()

In [16]:
def open_file(fname):
    f = open_(fname)
    try:
        yield f
    finally:
        f.close()

In [17]:
ctx = open_file('example.csv')

f = next(ctx) #opens the file and yields it

try:
    #do work with file
    pass
finally:
    try:
        next(ctx) #closes the file
    except StopIteration:
        pass


The above is still quite clunky and requires alot of code. We can write a class to implement a better approach.

In [18]:
class GenContext:
    def __init__(self, gen):
        self.gen = gen
        
    def __enter__(self):
        obj = next(self.gen) # opens the file
        return obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self.gen) #closes the file
        except StopIteration:
            pass
        return False

In [19]:
gen = open_file('example.csv')

with GenContext(gen) as f:
    pass

### Decorators as Context Managers

In the above example for using generator functions with context managers, there are a 2 downsides. One, we must call the generator function first (if it has arguments) and then pass it to the context manager class. Second, we lose understanding of what the context manager is and exactly what generator is it working with. 

__Using a Decorator__

In [20]:
class GenContextManager:
    def __init__(self, gen_obj):
        self.gen = gen_obj
        
    def __enter__(self):
        return next(self.gen)
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        try:
            next(self.gen)
        except StopIteration: ...
        return False

In [26]:
def contextmanager(gen_fn):
    def handler(*args, **kwargs):
        gen = gen_fn(*args, **kwargs)
        return GenContextManager(gen)
    
    return handler

In [27]:
@contextmanager
def open_file_gen(filename):
    f = open_(filename)
    try:
        yield f
    finally:
        f.close()

In [28]:
with open_file_gen('example.csv') as f:
    # do something with f
    ...

__The `contextlib` Module__

One of the goals when context managers were introduced to Python was to ensure generator functions could be used to easily create them (PEP343).

The `contextlib` module provides for us a context manager decorator that replicates what we did manually above, but it it a bit more complex and has better exception handling.

In [29]:
from contextlib import contextmanager